In [ ]:
# Importamos librerías

import numpy as np
import pandas as pd

# Abrimos fichero
# Introducir en path_file la ruta donde hemos descargado el fichero

path_file = "XXXXX"
df = pd.read_csv(path_file)

In [38]:
# Importamos librerías

from sklearn.model_selection import train_test_split

# Dividimos en train y test
df_train, df_test = train_test_split(df, test_size=0.25, shuffle=True, random_state=17)

In [39]:
# Eliminamos variables que no vamos a usar: 
    
    # "trans_date_trans_time", "date" y "time" por ser redundantes con el resto de variables date/time
    # "cc_num" por no aportar información relevante
    # "city", "state" y "zip" por ser redundantes con las variables 'lat' y 'long'

vars_todrop = ["trans_date_trans_time", "cc_num", "city", "state", "zip", "date", "time"]
df_train = df_train.drop(vars_todrop, axis=1)

# Creamos las variables "dist_lat" y "dist_long" como medidas de distancia del lugar de la transacción respecto al domicilio del titular de la tarjeta

df_train["dist_lat"] = df_train["lat"] - df_train["merch_lat"]
df_train["dist_long"] = df_train["long"] - df_train["merch_long"]

In [40]:
# Visualizamos NA's y NULL's

df_train.isna().sum()

# Procesamos los NA's de las variables "state_abbr" y "Unemployed percent":
    # En el caso de "state_abbr" con la categoría "NoNamed"
    # En el caso de "Unemployed percent" con su valor promedio

df_train['state_abbr'].fillna("NoNamed", inplace = True)
df_train['Unemployed percent'].fillna(df_train['Unemployed percent'].mean(), inplace = True)

In [41]:
# Transformamos las variables categóricas en one-hot

categorical_vars = [col for col in df_train.columns if df_train[col].dtype == 'object']
df_train = pd.get_dummies(df_train, columns=categorical_vars)

In [42]:
# Añadimos una categoría "_Others" a cada una de las variables categóricas. Como veremos más abajo servirá para garantizar presencia de las mismas variables en train y test, asimilando ambos conjuntos

for var in categorical_vars:
    df_train[var+"_Others"] = 0

In [ ]:
#Instalamos e importamos librerías

!pip install imbalanced-learn
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler

# Corregimos el desbalanceo de la variable objetivo "is_fraud" haciendo undersampling, buscamos un equilibrio: buen aprendizaje de la categoría minoritaria ('is_fraud' = 1) sin descuidar el aprendizaje de la categoría mayoritaria ('is_fraud' = 0)

X = df_train.drop('is_fraud', axis = 1)
y = df_train['is_fraud']

resampler = RandomUnderSampler(sampling_strategy ='auto', random_state=17)
X_resampled, y_resampled = resampler.fit_resample(X, y)

df_train = pd.DataFrame(X_resampled, columns = X.columns)
df_train['is_fraud'] = y_resampled

In [44]:
# Hacemos todas las transformaciones de train en el fichero de test

# Eliminamos variables que no vamos a usar

df_test = df_test.drop(vars_todrop, axis=1)

# Creamos las variables "dist_lat" y "dist_long"

df_test["dist_lat"] = df_test["lat"] - df_test["merch_lat"]
df_test["dist_long"] = df_test["long"] - df_test["merch_long"]

# Procesamos los NA's de las variables "state_abbr" y "Unemployed percent"

df_test['state_abbr'].fillna("NoNamed", inplace = True)
df_test['Unemployed percent'].fillna(df_test['Unemployed percent'].mean(), inplace = True)

# Transformamos las variables categóricas en one-hot

df_test = pd.get_dummies(df_test, columns=categorical_vars)

In [45]:
# Apoyándonos en las categorías "var+_Others" (añadidas más arriba) creamos, eliminamos y transformamos variables para asimilar los conjuntos de train y test

vars_no_incommon_train = [var for var in df_train.columns if var not in df_test.columns]
vars_no_incommon_test = [var for var in df_test.columns if var not in df_train.columns]

for var in vars_no_incommon_train:
    df_test[var] = 0
for var in vars_no_incommon_test:
    df_test = df_test.drop(var, axis = 1)
    df_test[var+"_Others"] = 1

df_train_columnssorted = df_train.columns
df_train = df_train[df_train_columnssorted]
df_test = df_test[df_train_columnssorted]

In [46]:
# Normalizamos los datos

# Dividimos entre X (variables predictoras) e y (variable a predecir)

X_train = df_train.drop('is_fraud', axis = 1)
y_train = df_train['is_fraud']

# Subdividimos X_train entre variables a normalizar y las que no normalizaremos

variables_tonorm = ['amt', 'lat', 'long', 'city_pop', 'merch_lat', 'merch_long', 'Year', 'Month', 'Unemployed percent', 'change_usd_eur', 'age', 'week_number', 'day', 'hour', 'crime_rate', 'dist_lat', 'dist_long']
X_train_tonorm = X_train[variables_tonorm]
X_train_notnorm = X_train.drop(variables_tonorm, axis = 1)

# Después normalizamos

from sklearn.preprocessing import StandardScaler
scaler_X = StandardScaler().fit(X_train_tonorm)
X_train_normalized = scaler_X.transform(X_train_tonorm)

# Por último volvemos a unir el X_train con las variables normalizadas y las no-normalizadas

X_train_notnorm = X_train_notnorm.values
X_train = np.concatenate([X_train_normalized, X_train_notnorm], axis = 1)

In [47]:
# Hacemos lo mismo en test, con la normalización (scaler) de train

X_test = df_test.drop('is_fraud', axis = 1)
y_test = df_test['is_fraud']

X_test_tonorm = X_test[variables_tonorm]
X_test_notnorm = X_test.drop(variables_tonorm, axis = 1)

X_test_normalized = scaler_X.transform(X_test_tonorm)

X_test_notnorm = X_test_notnorm.values
X_test = np.concatenate([X_test_normalized, X_test_notnorm], axis = 1)

In [48]:
# Importamos librerías

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score,roc_auc_score, roc_curve, f1_score

# Modelo 1: árbol de decisión sencillo

clf = DecisionTreeClassifier(random_state=17)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
TP = cm[1, 1]
FN = cm[1, 0]
FP = cm[0, 1]
TN = cm[0, 0]
print("Accuracy del modelo clf:", accuracy)
print("Precisión del modelo clf:", precision)
print("Recall del modelo clf:", recall)
print("Verdaderos Positivos:", TP)
print("Falsos Negativos:", FN)
print("Falsos Positivos:", FP)
print("Verdaderos Negativos:", TN)

Accuracy del modelo clf: 0.9641588515630567
Precisión del modelo clf: 0.12376211407085738
Recall del modelo clf: 0.9782335705316032
Verdaderos Positivos: 2337
Falsos Negativos: 52
Falsos Positivos: 16546
Verdaderos Negativos: 444164


In [49]:
# Importamos librerías

from sklearn.ensemble import RandomForestClassifier

# Modelo 2: random forest

rf = RandomForestClassifier(n_estimators = 200, random_state = 17)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
TP = cm[1, 1]
FN = cm[1, 0]
FP = cm[0, 1]
TN = cm[0, 0]
print("Accuracy del modelo clf:", accuracy)
print("Precisión del modelo clf:", precision)
print("Recall del modelo clf:", recall)
print("Verdaderos Positivos:", TP)
print("Falsos Negativos:", FN)
print("Falsos Positivos:", FP)
print("Verdaderos Negativos:", TN)

Accuracy del modelo clf: 0.9794709122671394
Precisión del modelo clf: 0.19622738135882553
Recall del modelo clf: 0.9623273336123901
Verdaderos Positivos: 2299
Falsos Negativos: 90
Falsos Positivos: 9417
Verdaderos Negativos: 451293


### A modo de conclusión

Los modelos buscan encontrar cierto equilibrio entre recall (como métrica base) y accuracy, capaces de detectar la práctica totalidad del fraude sin que se disparen los falsos positivos que marcarían como fraudulentas transacciones que no lo son.

En este sentido ambos modelos consiguen alcanzar ese equilibrio con métricas más que aceptables.